In [ ]:



import pandas as pd
from selenium import webdriver
from selenium.common import TimeoutException, NoSuchElementException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.firefox import GeckoDriverManager
from timeit import default_timer as timer
import time



start_timer = timer()


BASE_URL = 'https://www.congress.gov/u/qfqynCbSYlZHeuZXDIwFu'

def main():
        
    try:
        
        
        driver = webdriver.Firefox(service=Service(executable_path=GeckoDriverManager().install()))

        df = pd.DataFrame(columns=['Titles', 'Description'])
        
        i = 1
        driver.get(BASE_URL)
        while True:
            try:
                
                search_container = driver.find_element(By.ID, 'main')
                search_results_title = search_container.find_elements(By.XPATH,
                                                            "//li[@class='expanded']//span[@class='result-heading']")
                search_results_description = search_container.find_elements(By.XPATH, "//li[@class='expanded']\
                                                                                       //span[@class='result-title']")
                
                titles = {'Titles': [search_result.text for search_result in search_results_title]}
                description = {'Description': [search_result.text for search_result in search_results_description]}
                
                titles.update(description)
                
                
                #df = df.append(pd.DataFrame.from_dict(titles), ignore_index=True)
                df = pd.concat([df, pd.DataFrame.from_dict(titles)], ignore_index=True)

                df.to_excel('law_names.xlsx', engine='openpyxl', index=False)
                #print(df)
                try:
                    print("\rTotal time: {:.2f}".format((timer() - start_timer)/60.0 ), end='', flush=True)
                    next_button = driver.find_element(By.XPATH, "/html/body/div[2]/div/main/div/div/div[2]/div[2]/div[2]/a/i")
                    next_button.click()
                    i += 1
                    time.sleep(5)  # Wait for the next page to load
                except NoSuchElementException:
                    break
                
            except:
                print("break")
                break     
            
         
            
    except TimeoutException:
        print("Timed out waiting for data")
    except NoSuchElementException as e:
        print(e.msg)
    finally:
        df.to_excel('law_names_final.xlsx', engine='openpyxl', index= False)
        print(f'Number of titles found: {len(df)}')
        driver.quit()
  


if __name__ == "__main__":
    main()






